<a href="https://colab.research.google.com/github/lkhok22/ML-hw4/blob/main/face-recognition-betterCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install kaggle

In [3]:
import wandb
wandb.login()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lkhok22 (lkhok22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 89% 253M/285M [00:00<00:00, 759MB/s] 
100% 285M/285M [00:00<00:00, 791MB/s]


In [4]:
import pandas as pd
import numpy as np

In [5]:
train_df_t = pd.read_csv('train.csv')
test_df_t = pd.read_csv('test.csv')

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import random
import os

# Set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()


In [7]:
from PIL import Image

class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)
        label = int(row['emotion']) if 'emotion' in row else -1
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label


In [8]:
from sklearn.model_selection import train_test_split

# Split training data for validation
train_df, val_df = train_test_split(train_df_t, test_size=0.1, stratify=train_df_t['emotion'], random_state=42)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FERDataset(train_df, transform=transform)
val_dataset = FERDataset(val_df, transform=transform)

# Use very small subset (e.g. 20 images) to test overfitting
small_train_dataset = torch.utils.data.Subset(train_dataset, list(range(20)))

small_loader = DataLoader(small_train_dataset, batch_size=4, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [9]:
class BetterCNN(nn.Module):
    def __init__(self):
        super(BetterCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 24x24

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 12x12

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)   # 6x6
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 7)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [13]:
def train_model(model, loader, optimizer, criterion, epochs=10, use_wandb=False):
    model.to(device)  # ⬅️ Move model to GPU
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)  # ⬅️ Move data to GPU

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        acc = correct / total
        avg_loss = total_loss / len(loader)

        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Accuracy: {acc:.4f}")

        if use_wandb:
            wandb.log({"loss": avg_loss, "accuracy": acc})


In [12]:
import torch

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [14]:
wandb.init(project="fer-overfit-test", name="BetterCNN-overfit-small")

model = BetterCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, small_loader, optimizer, criterion, epochs=20, use_wandb=True)

wandb.finish()


Epoch 1/20 | Loss: 4.2145 | Accuracy: 0.1500
Epoch 2/20 | Loss: 2.1742 | Accuracy: 0.2000
Epoch 3/20 | Loss: 1.5961 | Accuracy: 0.4000
Epoch 4/20 | Loss: 1.4719 | Accuracy: 0.6500
Epoch 5/20 | Loss: 1.3301 | Accuracy: 0.5000
Epoch 6/20 | Loss: 1.3531 | Accuracy: 0.5500
Epoch 7/20 | Loss: 0.9608 | Accuracy: 0.6500
Epoch 8/20 | Loss: 0.9987 | Accuracy: 0.7000
Epoch 9/20 | Loss: 0.6203 | Accuracy: 0.8500
Epoch 10/20 | Loss: 0.4705 | Accuracy: 0.8000
Epoch 11/20 | Loss: 0.6030 | Accuracy: 0.7000
Epoch 12/20 | Loss: 0.2406 | Accuracy: 1.0000
Epoch 13/20 | Loss: 0.4811 | Accuracy: 0.9000
Epoch 14/20 | Loss: 0.3577 | Accuracy: 0.8500
Epoch 15/20 | Loss: 0.4631 | Accuracy: 0.8500
Epoch 16/20 | Loss: 0.1881 | Accuracy: 0.9500
Epoch 17/20 | Loss: 0.1092 | Accuracy: 1.0000
Epoch 18/20 | Loss: 0.1335 | Accuracy: 1.0000
Epoch 19/20 | Loss: 0.2276 | Accuracy: 0.9500
Epoch 20/20 | Loss: 0.3230 | Accuracy: 0.9000


accuracy,▁▁▃▅▄▄▅▆▇▆▆█▇▇▇████▇
loss,█▅▄▃▃▃▂▃▂▂▂▁▂▁▂▁▁▁▁▁
accuracy,0.9
loss,0.32301


In [15]:
wandb.init(project="fer-full-training", name="BetterCNN-full")

model = BetterCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, optimizer, criterion, epochs=20, use_wandb=True)

wandb.finish()


Epoch 1/20 | Loss: 1.7701 | Accuracy: 0.2859
Epoch 2/20 | Loss: 1.5462 | Accuracy: 0.3860
Epoch 3/20 | Loss: 1.4602 | Accuracy: 0.4274
Epoch 4/20 | Loss: 1.3964 | Accuracy: 0.4509
Epoch 5/20 | Loss: 1.3510 | Accuracy: 0.4676
Epoch 6/20 | Loss: 1.3174 | Accuracy: 0.4845
Epoch 7/20 | Loss: 1.2793 | Accuracy: 0.5026
Epoch 8/20 | Loss: 1.2492 | Accuracy: 0.5084
Epoch 9/20 | Loss: 1.2084 | Accuracy: 0.5258
Epoch 10/20 | Loss: 1.1714 | Accuracy: 0.5457
Epoch 11/20 | Loss: 1.1456 | Accuracy: 0.5497
Epoch 12/20 | Loss: 1.1097 | Accuracy: 0.5655
Epoch 13/20 | Loss: 1.0777 | Accuracy: 0.5755
Epoch 14/20 | Loss: 1.0357 | Accuracy: 0.5911
Epoch 15/20 | Loss: 1.0047 | Accuracy: 0.6034
Epoch 16/20 | Loss: 0.9777 | Accuracy: 0.6125
Epoch 17/20 | Loss: 0.9473 | Accuracy: 0.6250
Epoch 18/20 | Loss: 0.9166 | Accuracy: 0.6375
Epoch 19/20 | Loss: 0.8919 | Accuracy: 0.6423
Epoch 20/20 | Loss: 0.8712 | Accuracy: 0.6533


accuracy,▁▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇███
loss,█▆▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
accuracy,0.65334
loss,0.87121


In [16]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {correct / total:.4f}")


In [17]:
evaluate(model, val_loader)

Validation Accuracy: 0.5691
